### DATA CLEANING PYTHON SCRIPT FOR ANALYZING GLOBAL COVID 19 VACCINE  DISTRIBUTION

### Importing libraries

In [1]:
# If module not found error appears then please install the repected library first

import pandas as pd
import numpy as np
import json
import math
from urllib.request import urlopen
import datetime

### Data Loading, Transformation, and Cleaning

In [2]:
# Getting different countries name and iso code from the csv file

locations = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/locations.csv')
locations = locations[['location', 'iso_code']]
locations.head()

,location,iso_code
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,Andorra,AND
4,Angola,AGO


In [3]:
# Getting vaccinations timeseries data

vaccinations = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv')
vaccinations.head()

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0


In [4]:
# Creating new dataframe by left joining the locations and vaccinations dataframes

df = locations.merge(vaccinations, how='left', on='iso_code')
del df['location_x']
df = df.rename(columns = {'location_y' : 'location'})
df.head()

,iso_code,location,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million
0,AFG,Afghanistan,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN
1,AFG,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0
2,AFG,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0
3,AFG,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0
4,AFG,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0


In [5]:
# Getting data for number of vaccine doses given of each manufacturer 
# for each country datewise

vacc_by_mfg = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations-by-manufacturer.csv')
vacc_by_mfg = vacc_by_mfg.rename(columns = {'total_vaccinations' : 'total_vaccinations_by_mfg'})
vacc_by_mfg.head()

,location,date,vaccine,total_vaccinations_by_mfg
0,Chile,2020-12-24,Pfizer/BioNTech,420
1,Chile,2020-12-25,Pfizer/BioNTech,5198
2,Chile,2020-12-26,Pfizer/BioNTech,8338
3,Chile,2020-12-27,Pfizer/BioNTech,8649
4,Chile,2020-12-28,Pfizer/BioNTech,8649


In [6]:
# Left joining the data on location and date

df = df.merge(vacc_by_mfg, how='left', on=['location', 'date'])
df.head()

,iso_code,location,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccine,total_vaccinations_by_mfg
0,AFG,Afghanistan,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,NaN,NaN
2,AFG,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,NaN,NaN
3,AFG,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,NaN,NaN
4,AFG,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,NaN,NaN


In [7]:
# Creating last_updated column to keep track of the date on which
# the data for each country was updated

last_updated = dict(df[['iso_code','date']].groupby(by='iso_code').max().reset_index().values)
df['last_updated'] = df['iso_code']
df['last_updated'] = df['last_updated'].map(lambda x: last_updated.get(x,''))
df.head()

,iso_code,location,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccine,total_vaccinations_by_mfg,last_updated
0,AFG,Afghanistan,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,2021-05-11
1,AFG,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,NaN,NaN,2021-05-11
2,AFG,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,NaN,NaN,2021-05-11
3,AFG,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,NaN,NaN,2021-05-11
4,AFG,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,NaN,NaN,2021-05-11


In [8]:
# MAX_DATE is the tomorrow's date as python range function does not include last value

MAX_DATE = pd.to_datetime("today") + datetime.timedelta(days=1)
MAX_DATE = str(MAX_DATE.date())

# List of iso codes for each country

iso = df['iso_code'].unique().tolist()

In [9]:
# Filling the numbers till today's date for each country to bring standardization

final_df_values = []

for i in iso:
    temp_df = df[df['iso_code'] == i]
    START_DATE = temp_df['date'].max()
    dates = pd.date_range(start=START_DATE, end=MAX_DATE).astype(str).tolist()
    dates.pop(0)
    values = temp_df.values.tolist()
    for j in range(0,len(dates)):
        v = temp_df.values[-1].tolist()
        v[2] = dates[j]
        values.append(v)
    final_df_values.extend(values)


In [10]:
# Creating dataframe from updated list values 

df = pd.DataFrame(final_df_values, columns=df.columns)
df

,iso_code,location,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccine,total_vaccinations_by_mfg,last_updated
0,AFG,Afghanistan,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN,2021-05-11
1,AFG,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,NaN,NaN,2021-05-11
2,AFG,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,NaN,NaN,2021-05-11
3,AFG,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,NaN,NaN,2021-05-11
4,AFG,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,NaN,NaN,2021-05-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23038,ZWE,Zimbabwe,2021-05-17,814258.0,591808.0,222450.0,17311.0,17658.0,5.48,3.98,1.50,1188.0,NaN,NaN,2021-05-19
23039,ZWE,Zimbabwe,2021-05-18,831954.0,600579.0,231375.0,17696.0,17455.0,5.60,4.04,1.56,1174.0,NaN,NaN,2021-05-19
23040,ZWE,Zimbabwe,2021-05-19,844684.0,605556.0,239128.0,12730.0,16331.0,5.68,4.07,1.61,1099.0,NaN,NaN,2021-05-19
23041,ZWE,Zimbabwe,2021-05-20,844684.0,605556.0,239128.0,12730.0,16331.0,5.68,4.07,1.61,1099.0,NaN,NaN,2021-05-19


In [11]:
# Forward filling values in the following columns for each country

for i in iso:
    temp_df = df[df['iso_code'] == i]
    temp_df['total_vaccinations'] = temp_df['total_vaccinations'].ffill()
    temp_df['people_vaccinated'] = temp_df['people_vaccinated'].ffill()
    temp_df['people_fully_vaccinated'] = temp_df['people_fully_vaccinated'].ffill()
    temp_df['total_vaccinations_per_hundred'] = temp_df['total_vaccinations_per_hundred'].ffill()
    temp_df['people_vaccinated_per_hundred'] = temp_df['people_vaccinated_per_hundred'].ffill()
    temp_df['people_fully_vaccinated_per_hundred'] = temp_df['people_fully_vaccinated_per_hundred'].ffill()
    df[df['iso_code'] == i] = temp_df


<ipython-input-11-ddfe474c654d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['total_vaccinations'] = temp_df['total_vaccinations'].ffill()
<ipython-input-11-ddfe474c654d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['people_vaccinated'] = temp_df['people_vaccinated'].ffill()
<ipython-input-11-ddfe474c654d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [12]:
# Replacing 0 with Nan as 0 will be inferred as no doses were given on a particular
# date which may not be true

df = df.replace({0 : math.nan})

In [13]:
# Updating daily measures' numbers based on the cumulative measures' numbers after filling 
# empty values, for our data to be accurate

values = df.values

for i in range(0,len(df)):
    if(values[i][3]==values[i-1][3]):
        values[i][7] = 0
        values[i][11] = values[i-1][11]
    if(values[i][3]==values[i-1][3]):
        values[i][8] = values[i-1][8]
    if(values[i][4]==values[i-1][4]):
        values[i][9] = values[i-1][9]
    if(values[i][5]==values[i-1][5]):
        values[i][10] = values[i-1][10]
            
df = pd.DataFrame(values)

In [14]:
# Giving columns some meaningful names

df.columns = ['iso_code','location', 'date', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated',
       'daily_vaccinations_raw', 'daily_vaccinations',
       'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred',
       'people_fully_vaccinated_per_hundred', 'daily_vaccinations_per_million',
       'vaccine', 'total_vaccinations_by_mfg','last_updated']

In [15]:
# Removing unnecessary column of no use

del df['daily_vaccinations_raw']
df.head()

,iso_code,location,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccine,total_vaccinations_by_mfg,last_updated
0,AFG,Afghanistan,2021-02-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-11
1,AFG,Afghanistan,2021-02-23,NaN,NaN,NaN,1367,NaN,NaN,NaN,35,NaN,NaN,2021-05-11
2,AFG,Afghanistan,2021-02-24,NaN,NaN,NaN,1367,NaN,NaN,NaN,35,NaN,NaN,2021-05-11
3,AFG,Afghanistan,2021-02-25,NaN,NaN,NaN,1367,NaN,NaN,NaN,35,NaN,NaN,2021-05-11
4,AFG,Afghanistan,2021-02-26,NaN,NaN,NaN,1367,NaN,NaN,NaN,35,NaN,NaN,2021-05-11


In [16]:
# Correcting any typo error if present in the data

df['people_fully_vaccinated_per_hundred'] = df['people_fully_vaccinated_per_hundred'].map(lambda x: 100 if(x>100) else x)
df['total_vaccinations_per_hundred'] = df['total_vaccinations_per_hundred'].map(lambda x: 100 if(x>100) else x)
df['people_vaccinated_per_hundred'] = df['people_vaccinated_per_hundred'].map(lambda x: 100 if(x>100) else x)

In [17]:
# Creating dictionary of countries and available vaccines
# which can be used to create a new column vaccines_availabe

vaccines_available_df = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/locations.csv')
vaccines_available_df = vaccines_available_df[['iso_code','vaccines']]
vaccines_dict = dict(vaccines_available_df.values.tolist())

df['vaccines_available'] = df['iso_code']
df['vaccines_available'] = df['iso_code'].map(lambda x : vaccines_dict.get(x))
df.head()

,iso_code,location,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccine,total_vaccinations_by_mfg,last_updated,vaccines_available
0,AFG,Afghanistan,2021-02-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-11,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm..."
1,AFG,Afghanistan,2021-02-23,NaN,NaN,NaN,1367,NaN,NaN,NaN,35,NaN,NaN,2021-05-11,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm..."
2,AFG,Afghanistan,2021-02-24,NaN,NaN,NaN,1367,NaN,NaN,NaN,35,NaN,NaN,2021-05-11,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm..."
3,AFG,Afghanistan,2021-02-25,NaN,NaN,NaN,1367,NaN,NaN,NaN,35,NaN,NaN,2021-05-11,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm..."
4,AFG,Afghanistan,2021-02-26,NaN,NaN,NaN,1367,NaN,NaN,NaN,35,NaN,NaN,2021-05-11,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm..."


In [18]:
# Creating new columns for each vaccine with values 0 or 1
# 0 means particular vaccine is not available in the country
# 1 means particular vaccine is available in the country

df['Oxford/AstraZeneca'] = df['vaccines_available'].map(lambda x : 1 if('Oxford/AstraZeneca' in x) else 0);
df['Pfizer/BioNTech'] = df['vaccines_available'].map(lambda x : 1 if('Pfizer/BioNTech' in x) else 0);
df['Sinovac'] = df['vaccines_available'].map(lambda x : 1 if('Sinovac' in x) else 0);
df['Sputnik'] = df['vaccines_available'].map(lambda x : 1 if('Sputnik' in x) else 0);
df['Sinopharm/Beijing'] = df['vaccines_available'].map(lambda x : 1 if('Sinopharm/Beijing' in x) else 0);
df['Moderna'] = df['vaccines_available'].map(lambda x : 1 if('Moderna' in x) else 0);
df['Covaxin'] = df['vaccines_available'].map(lambda x : 1 if('Covaxin' in x) else 0);
df['Johnson&Johnson'] = df['vaccines_available'].map(lambda x : 1 if('Johnson&Johnson' in x) else 0);

In [19]:
# Getting continent csv file to assign continent name to each country

continent_df = pd.read_csv('https://raw.githubusercontent.com/datawookie/data-diaspora/master/spatial/country-continent-codes.csv')
continent_df.head()

,,,,,# https://datahub.io/JohnSnowLabs/country-and-continent-codes-list
continent,code,country,iso2,iso3,number
Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4
Europe,EU,"Albania, Republic of",AL,ALB,8
Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10
Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12


In [20]:
# Creating dictionary of countries and continents to map respected continent
# to each country

continents =  [i[0] for i in continent_df.index.tolist()][1:]
codes =  [i[4] for i in continent_df.index.tolist()][1:]
count_cont_dict = dict(zip(codes,continents))

df['continent'] = df['iso_code']
df['continent'] = df['continent'].map(lambda x : count_cont_dict.get(x))

In [21]:
# Exporting the final data in MS Excel Workbook format

df.to_excel('clean_data.xlsx',index=False)